In [3]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
tf.__version__

'2.12.0'

In [5]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2023-09-06 06:24:02--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7757635 (7.4M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.40M  6.26MB/s    in 1.2s    

2023-09-06 06:24:04 (6.26 MB/s) - ‘fra-eng.zip’ saved [7757635/7757635]



In [6]:
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [7]:
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  return ''.join(c for c in unicodedata.normalize('NFD', s)
  if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가
  # ex) 'I am a student.' -> 'I am a student .'
  sent = re.sub(r'([?.!,?`])', r' \1', sent)

  # (a-z, A-Z, '.', '?', '!', ',')이들을 제외하고 전부공백으로 변환
  sent = re.sub(r'[^a-zA-A!.?]+', r' ', sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r'\s+', ' ', sent)
  return sent

In [8]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장: ', en_sent)
print('전처리 영어 문장: ', preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장: ', fr_sent)
print('전처리 후 프랑스어 문장: ', preprocess_sentence(fr_sent))

전처리 전 영어 문장:  Have you had dinner?
전처리 영어 문장:  have you had dinner ?
전처리 전 프랑스어 문장:  Avez-vous déjà diné?
전처리 후 프랑스어 문장:  avez vous deja dine ?


In [9]:
num_samples = 33000

In [10]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open('fra.txt', 'r') as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ('<sos> ' + tar_line).split()]
      tar_line_out = [w for w in (tar_line + ' <eos>').split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [11]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력: ', sents_en_in[:5])
print('디코더의 입력: ', sents_fra_in[:5])
print('디코더의 레이블: ', sents_fra_out[:5])

인코더의 입력:  [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력:  [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블:  [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [12]:
tokenizer_en = Tokenizer(filters = '',lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding = 'post')

tokenizer_fra = Tokenizer(filters = '', lower = False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding = 'post')

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding = 'post')

In [13]:
print('인코더의 입력 크기(shape): ', encoder_input.shape)
print('디코더의 입력 크기(shape): ', decoder_input.shape)
print('디코더의 레이블의 크기(shape): ', decoder_target.shape)

인코더의 입력 크기(shape):  (33000, 7)
디코더의 입력 크기(shape):  (33000, 16)
디코더의 레이블의 크기(shape):  (33000, 16)


In [14]:
tokenizer_en.word_index

{'.': 1,
 'i': 2,
 'you': 3,
 '?': 4,
 'tom': 5,
 'it': 6,
 's': 7,
 'is': 8,
 'a': 9,
 're': 10,
 'we': 11,
 'm': 12,
 't': 13,
 'that': 14,
 'he': 15,
 'the': 16,
 'me': 17,
 'this': 18,
 'are': 19,
 'was': 20,
 'do': 21,
 'to': 22,
 '!': 23,
 'they': 24,
 'can': 25,
 'go': 26,
 'don': 27,
 'have': 28,
 'my': 29,
 'your': 30,
 'not': 31,
 'll': 32,
 'be': 33,
 'what': 34,
 'here': 35,
 'did': 36,
 'no': 37,
 'let': 38,
 'all': 39,
 'she': 40,
 'like': 41,
 'in': 42,
 'get': 43,
 'how': 44,
 'up': 45,
 'on': 46,
 'need': 47,
 'very': 48,
 'who': 49,
 'one': 50,
 'love': 51,
 'want': 52,
 'come': 53,
 'out': 54,
 'him': 55,
 'got': 56,
 'please': 57,
 'look': 58,
 'now': 59,
 'so': 60,
 'there': 61,
 'know': 62,
 'help': 63,
 'stop': 64,
 'us': 65,
 'too': 66,
 'just': 67,
 'take': 68,
 've': 69,
 'see': 70,
 'will': 71,
 'were': 72,
 'good': 73,
 'of': 74,
 'has': 75,
 'for': 76,
 'am': 77,
 'give': 78,
 'where': 79,
 'at': 80,
 'must': 81,
 'had': 82,
 'work': 83,
 'keep': 84,
 'back

In [15]:
tokenizer_fra.word_index

{'.': 1,
 '<sos>': 2,
 '<eos>': 3,
 'je': 4,
 'est': 5,
 '?': 6,
 'tom': 7,
 'a': 8,
 'vous': 9,
 '!': 10,
 'pas': 11,
 'il': 12,
 'j': 13,
 'nous': 14,
 'le': 15,
 'ai': 16,
 'tu': 17,
 'de': 18,
 'c': 19,
 'ne': 20,
 'suis': 21,
 'la': 22,
 'l': 23,
 'un': 24,
 'en': 25,
 'ce': 26,
 'n': 27,
 'etes': 28,
 'ca': 29,
 'me': 30,
 'que': 31,
 'une': 32,
 'les': 33,
 'moi': 34,
 'es': 35,
 'd': 36,
 's': 37,
 'sommes': 38,
 't': 39,
 'y': 40,
 'sont': 41,
 'elle': 42,
 'etait': 43,
 'm': 44,
 'ils': 45,
 'fait': 46,
 'qui': 47,
 'tout': 48,
 'as': 49,
 'aime': 50,
 'elles': 51,
 'des': 52,
 'mon': 53,
 'te': 54,
 'bien': 55,
 'toi': 56,
 'ici': 57,
 'avez': 58,
 'du': 59,
 'ete': 60,
 'tres': 61,
 'besoin': 62,
 'peux': 63,
 'faire': 64,
 'avons': 65,
 'va': 66,
 'se': 67,
 'qu': 68,
 'cela': 69,
 'on': 70,
 'air': 71,
 'votre': 72,
 'faut': 73,
 'fais': 74,
 'personne': 75,
 'veux': 76,
 'etais': 77,
 'ont': 78,
 'comment': 79,
 'ton': 80,
 'aller': 81,
 'monde': 82,
 'tous': 83,
 'trop'

In [16]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print('영어 단어집합의 크기: {:d}, 프랑스어 단어 집합의 크기: {:d}'.format(src_vocab_size, tar_vocab_size))

영어 단어집합의 크기: 4481, 프랑스어 단어 집합의 크기: 7873


In [17]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [18]:
max_src_len = encoder_input.shape[1]
max_tar_len = decoder_input.shape[1]
print('source 문장의 최대 길이: ', max_src_len)
print('target 문장의 최대 길이: ', max_tar_len)

source 문장의 최대 길이:  7
target 문장의 최대 길이:  16


In [19]:
encoder_input[0]

array([26,  1,  0,  0,  0,  0,  0], dtype=int32)

In [20]:
decoder_input[138]

array([  2,  19,   5, 954,  18, 893,  10,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [21]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덥 시퀀스: ', indices)

랜덥 시퀀스:  [23550 28181  5642 ...  6193 29850 20515]


In [22]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [23]:
encoder_input[138]

array([38, 17, 26,  1,  0,  0,  0], dtype=int32)

In [24]:
decoder_input[138]

array([  2, 132,  34, 107,  10,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [25]:
decoder_target[138]

array([132,  34, 107,  10,   3,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [26]:
n_of_val = int(33000 * 0.1)
print('검증 데이터의 개수: ', n_of_val)

검증 데이터의 개수:  3300


In [27]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [28]:
print('훈련 source 데이터의 크기: ', encoder_input_train.shape)
print('훈련 target 데이터의 크기: ', decoder_input_train.shape)
print('훈련 target 레이블의 크기: ', decoder_target_train.shape)
print('테스트 source 데이터의 크기: ', encoder_input_test.shape)
print('테스트 target 데이터의 크기: ', decoder_input_test.shape)
print('테스트 target 레이블의 크기: ', decoder_target_test.shape)

훈련 source 데이터의 크기:  (29700, 7)
훈련 target 데이터의 크기:  (29700, 16)
훈련 target 레이블의 크기:  (29700, 16)
테스트 source 데이터의 크기:  (3300, 7)
테스트 target 데이터의 크기:  (3300, 16)
테스트 target 레이블의 크기:  (3300, 16)


In [29]:
from tensorflow.keras.layers import Input,LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [30]:
embedding_dim = 64
hidden_units = 64

In [31]:
# 인코더
encoder_inputs = Input(shape = (None, ))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value = 0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state = True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [32]:
# 디코더
decoder_inputs = Input(shape = (None, ))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value = 0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_state는 True
decoder_lstm = LSTM(hidden_units, return_sequences = True, return_state = True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state = encoder_states)
# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [33]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 64)     286784      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 64)     503872      ['input_2[0][0]']                
                                                                                              

In [36]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test],
                             decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 41s 117ms/step - loss: 3.3327 - acc: 0.6146 - val_loss: 1.9632 - val_acc: 0.6243
Epoch 2/50
233/233 [==============================] - 8s 36ms/step - loss: 1.7988 - acc: 0.7025 - val_loss: 1.7033 - val_acc: 0.7476
Epoch 3/50
233/233 [==============================] - 9s 40ms/step - loss: 1.6175 - acc: 0.7493 - val_loss: 1.5739 - val_acc: 0.7531
Epoch 4/50
233/233 [==============================] - 9s 37ms/step - loss: 1.4944 - acc: 0.7611 - val_loss: 1.4665 - val_acc: 0.7649
Epoch 5/50
233/233 [==============================] - 7s 31ms/step - loss: 1.3881 - acc: 0.7720 - val_loss: 1.3785 - val_acc: 0.7772
Epoch 6/50
233/233 [==============================] - 8s 35ms/step - loss: 1.2978 - acc: 0.7874 - val_loss: 1.3056 - val_acc: 0.7922
Epoch 7/50
233/233 [==============================] - 8s 33ms/step - loss: 1.2264 - acc: 0.7985 - val_loss: 1.2467 - val_acc: 0.7999
Epoch 8/50
233/233 [==============================] - 7s 30ms/step 

In [37]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape = (hidden_units, ))
decoder_state_input_c = Input(shape = (hidden_units, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2,
                                                    initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [38]:
# 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [39]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <sos>에 해당하는 정수 생성
  target_seq = np.zeros((1, 1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' ' + sampled_char

    # '<eos>'에 도달하거나 정해진 길이를 넘으면 중단
    if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
      stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [40]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and \
       encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [41]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print('입력 문장: ', seq_to_src(encoder_input_train[seq_index]))
  print('정답 문장: ', seq_to_tar(decoder_input_train[seq_index]))
  print('번역 문장: ', decoded_sentence[1: -5])
  print('-' * 50)

1/1 [==============================] - 0s 19ms/step
입력 문장:  do it by yourself . 
정답 문장:  fais le toi meme ! 
번역 문장:  fais le simplement . 
--------------------------------------------------
1/1 [==============================] - 0s 22ms/step
입력 문장:  we talked . 
정답 문장:  nous avons discute . 
번역 문장:  nous nous sommes entretenues . 
--------------------------------------------------
1/1 [==============================] - 0s 20ms/step
입력 문장:  i ll be back soon . 
정답 문장:  je serai vite de retour . 
번역 문장:  je serai au regime . 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력 문장:  here you go sir . 
정답 문장:  voila monsieur . 
번역 문장:  c est ici . 
--------------------------------------------------
1/1 [==============================] - 0s 24ms/step
입력 문장:  i came for you . 
정답 문장:  je suis venu a cause de toi . 
번역 문장:  je suis venu pour toi . 
--------------------------------------------------
